# Tensorflow multioutput model


In [ ]:
#@title ##**Prepare environment** { display-mode: "form" }

from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print("Environment prepared for work")

In [ ]:
#@title ##**Prepare dataset** { display-mode: "form" }

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(len(x_train), 28, 28, 1)
x_test = x_test.reshape(len(x_test), 28, 28, 1)

y_train_b = [0 if i % 2 else 1 for i in y_train]
y_test_b = [0 if i % 2 else 1 for i in y_test]

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train_b)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train_b = onehot_encoder.fit_transform(integer_encoded)

print(f'Dataset successfully prepared for work\n'\
    f'Dataset contains: {len(x_train) + len(x_test)} images')

In [ ]:
#@title ##**Show dataset examples** { display-mode: "form" }

plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i].reshape(28, 28), cmap=plt.cm.binary)
    label = str(y_train[i])
    val = "odd" if (y_train_b[i][0] == 1) else "even"
    label += ", " + val
    plt.xlabel(label)
plt.show()

# Make model


In [ ]:
main_input = tf.keras.Input(shape=(28, 28, 1), name='main_input')

x = tf.keras.layers.Convolution2D(32, (3, 3), input_shape = (28, 28, 1), activation='relu')(main_input)
x = tf.keras.layers.ZeroPadding2D((1, 1))(x)
x = tf.keras.layers.Convolution2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Flatten()(x)

numeric_out = tf.keras.layers.Dense(10, activation='softmax', name='numeric_out')(x)
binary_out = tf.keras.layers.Dense(2, activation='sigmoid', name='binary_out')(x)

model = tf.keras.Model(inputs=main_input, outputs=[numeric_out, binary_out])
model.compile(optimizer='rmsprop', loss=['sparse_categorical_crossentropy', 'binary_crossentropy'], metrics=['accuracy'])
model.summary()

In [ ]:
#@title ##**Plot model (optional)** { display-mode: "form" }

from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#@title ##**Train model** { display-mode: "form" }

ep = model.fit(x_train, { "numeric_out": y_train, "binary_out": y_train_b}, epochs=1)

In [ ]:
#@title ##**Evaluate model (optional)** { display-mode: "form" }

x = model.evaluate(x_train,  { "numeric_out": y_train, "binary_out": y_train_b}, verbose=2)

# Test model

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    img = x_train[i]
    pred = model.predict(img.reshape(1, 28, 28, 1))
    label = str(np.argmax(pred[0]))
    val = "even" if (np.argmax(pred[1]) == 1) else "odd"
    label += ", " + val
    
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img.reshape(28, 28), cmap=plt.cm.binary)
    
    plt.xlabel(label)
plt.show()